<a href="https://colab.research.google.com/github/gtc0823/RAG/blob/main/RAG02_%E6%89%93%E9%80%A0_RAG_%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 使用的向量資料庫檔案

原本是用爬蟲取PTT Cfantasy(網路小說) 版的資訊，大概900多筆文章，不過感覺是每篇文章內容差異過大，

chunk_size, chunk_overlap非常難設計，因此沒辦法準確的利用query搜尋正確的向量

所以後來只單純利用 https://www.ptt.cc/bbs/CFantasy/M.1690738559.A.30F.html 的文章內容

來製作向量資料庫，由於每篇小說的介紹大概都為500字，因此chunk_size設為500, chunk_overlap設為50

In [2]:
URL = "https://drive.google.com/uc?export=download&id=1JCU-dl0MFt9sXwJUu5wmgGv5WVRJjQ0J"

!wget -O faiss_db.zip "$URL"

--2025-04-12 08:49:04--  https://drive.google.com/uc?export=download&id=1JCU-dl0MFt9sXwJUu5wmgGv5WVRJjQ0J
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.138, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1JCU-dl0MFt9sXwJUu5wmgGv5WVRJjQ0J&export=download [following]
--2025-04-12 08:49:04--  https://drive.usercontent.google.com/download?id=1JCU-dl0MFt9sXwJUu5wmgGv5WVRJjQ0J&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49615 (48K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]  48.45K  --.-KB/s    in 0.00

In [3]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [4]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

CustomE5Embedding功能: 可以「透過 Langchain 的統一介面，使用 Hugging Face 上的語意模型」

In [7]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 `faiss_db`

並把 FAISS 向量庫轉成「可查詢」的檢索器 (retriever)

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 4. 設定好我們要的 LLM

In [9]:
import os
from google.colab import userdata

利用 OpenAI API。並使用免費的 Groq 服務

In [10]:
api_key = userdata.get('Groq')

In [11]:
os.environ["OPENAI_API_KEY"] = api_key

模型和 `base_url` 使用 Groq

In [12]:
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [13]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5. `prompt` 設計

In [17]:
system_prompt = '''
你是提供中文回應的網路小說討論小助手，只能用台灣習慣的中文回答。
請根據使用者的問題和你找到的資料，提供簡潔、有內容、親切又帶點幽默的回答"
'''

prompt_template = '''
下面是從 PTT 抓到的一些討論內容，可能有助於回答使用者的問題：
------------------------
{retrieved_chunks}
------------------------

使用者想問的問題是：
「{question}」

請根據上面資料回應，語氣自然一點，像在跟鄉民聊天一樣。
如果資料不足，就坦率地建議使用者自己去google，不要亂猜或硬掰。
'''

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

`docs = retriever.get_relevant_documents(user_input)`

Langchain 會自動：

1.   將查詢文字轉成 "query: ..." 並丟進嵌入模型
2.   拿轉出來的語意向量去和資料庫比對
3.   回傳最相似的文件片段（原始文件、metadata 都會帶回來）

In [18]:
###chat_history = []

def chat_with_rag(user_input, chat_history):
    ###global chat_history

    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 設定系統角色
    messages = [{"role": "system", "content": system_prompt}]

    # 將過往對話加入
    for question, answer in chat_history:
        messages.append({"role": "user", "content": question})
        messages.append({"role": "assistant", "content": answer})

    # 本輪對話加入
    messages.append({"role": "user", "content": final_prompt})

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
      model=model,
      messages=messages
    )
    answer = response.choices[0].message.content

    ###chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# AI網路小說討論助手")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message, chat_history_local)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)